BACnet Device Scanning
====================

While direct BACnet discovery techniques are occasionally useful, it is more common to use the NF scan API to initiate and retrieve results from BACnet scans.  When the scan service is in use, the Scan service takes are of finding devices, downloading object lists, and inserting them into the database.  `NF::BACnet` comes with a powerful set of scan options to help you find the most stubborn BACnet devices on your network with only a few commands.

In [1]:
import os
import grpc
from normalgw.bacnet import scan_pb2, scan_pb2_grpc, bacnet_pb2

In [2]:
# the environment in docker-compose has the right address to connect to the BACnet service on.  
# os.environ["SCAN_SERVICE_ADDRESS"] ="localhost:9091"
channel = grpc.insecure_channel("localhost:8080")
stub = scan_pb2_grpc.ScanStub(channel)

In [3]:
# basic discovery just uses a device discovery job
job = scan_pb2.StartJobRequest()

# add a target with defaults.  This does a global broadcast.  
#  You can add custom whois targets here, for instance to 
# implement a series of range scans.  they are run sequentially.
job.device.targets.extend([bacnet_pb2.WhoIsRequest()])

# if set to true, this enqueues Object scan jobs for each 
# discovered device to download the object list.
job.device.auto_scan = False
# import the results into the db.  if you just want to see what has 
# changed not importing can be a good idea.  You can import later using ImportJob
job.auto_import = False

started_job = stub.StartJob(job)
print(started_job)

id: 1
status_message: "Waiting to run"
timestamps {
  queued_at {
    seconds: 1630526950
    nanos: 437863784
  }
}
device {
  targets {
  }
}



In [5]:
# get the results of the jobs.  you have to send full=True to get the full object list
stub.GetJobs(scan_pb2.GetJobRequest(id_filter=started_job.id, full=True))

results {
  id: 1
  status: DONE
  status_message: "Device Scan Completed"
  timestamps {
    queued_at {
      seconds: 1630526950
      nanos: 437863784
    }
    run_at {
      seconds: 1630526950
      nanos: 439861662
    }
    finished_at {
      seconds: 1630526962
      nanos: 386042735
    }
  }
  device {
    targets {
      options {
        timeout: 1800.0
        priority: 10
        block_lower_priority: true
      }
    }
  }
  device_result {
    device_address {
      mac: "\300\250g\200\272\300"
      max_apdu: 1476
      device_id: 150
    }
  }
  device_result {
    device_address {
      mac: "\300\250g\222\272\300"
      max_apdu: 1476
      device_id: 260001
    }
    scanned_props {
      key: 12
      value {
        character_string: "0.9"
      }
    }
    scanned_props {
      key: 28
      value {
        character_string: "NF Stack"
      }
    }
    scanned_props {
      key: 44
      value {
        character_string: "1.0.0"
      }
    }
    scanned_pro

In [ ]:
job = scan_pb2.StartJobRequest()

# you can do a network scan.  this uses a more complicated algorithm 
# to find devices hidden behind routers on other subnets, that don't
# reply to Who-Is requests, or running on non-standard ports.

# you can provide a CIDR subnet here.  If you provide 0.0.0.0 as the 
# network number, the local network is used.
job.network.subnet = '0.0.0.0/24'

# scan multiple ports or ranges of ports to find devices not on the expected port.
job.network.ports = '47808'
job.network.auto_scan = True

started_job = stub.StartJob(job)
print(started_job)